<a href="https://colab.research.google.com/github/ErnestoAvedillo/AAM-math/blob/main/Reto_AAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [293]:
#!pip install --upgrade gspread

In [294]:
#!pip install --upgrade pandas
#!pip install --upgrade Lib/datetime.py

In [295]:
import pandas as pd
import numpy as np
from google.colab import auth
import gspread
from google.auth import default#autenticating to google
import datetime as dt
from datetime import timedelta

In [296]:
gspread.__version__

'3.4.2'

In [297]:
pd.__version__

'1.5.3'

In [298]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Reading Excel AAM data.gsheet


In [299]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
file_path = "/content/drive/MyDrive/Colab Notebooks/AAM data.gsheet"
try:
    #https://docs.google.com/spreadsheets/d/179giTQNZghst9cpCtAzmTpHt0DrOtRQiBplU6-_XiUU/edit?usp=drive_link
    AAM_workbook = gc.open_by_key('179giTQNZghst9cpCtAzmTpHt0DrOtRQiBplU6-_XiUU')
    #worksheet = gc.open(file_path)
    print("Google Sheet opened successfully.")
except gspread.exceptions.SpreadsheetNotFound:
    print("Error: Spreadsheet not found.") #Convert to a DataFrame


Google Sheet opened successfully.


# **Primera parte**

Modelado de la producción

Cálculo del coste de las demandas internas.

known_references = nḿero de referencias (n).

REF = diccionario con las referencias de la empresa (TODAS , las de compra , internas  y de producto acabado.)

DC = Demanda de cliente

BOM = es una tabla de n x n con el BOM de los producto (Todos) Build of Materials

##Obteniendo un listado de las referencias.


In [300]:
BOM_worksheet = AAM_workbook.worksheet('BOM')#Convert to a DataFrame
rows = BOM_worksheet.get_all_values()#Convert to a DataFrame
df = pd.DataFrame(rows)
#creating columns name
df.columns = df.iloc[0]
df = df.iloc[1:]
REF_df = pd.concat([df['Ref Entrada'],df['Ref Salida']]).drop_duplicates()
#REF = REF_df[:].to_dict(orient='dict') does not work will try again
REF = {}
n = 0
for i in REF_df:
  REF[i] = n
  n += 1
print ("Diccionario de referencias con sus índices")
print(REF)

Diccionario de referencias con sus índices
{'AAA': 0, 'AMA': 1, 'BAJ': 2, 'AAB': 3, 'CAT': 4, 'AAE': 5, 'BAC': 6, 'CAR': 7, 'AAF': 8, 'BAD': 9, 'CAV': 10, 'AAC': 11, 'CAB': 12, 'CAX': 13, 'CAH': 14}


In [301]:
# Obtengo el numero de referencias
known_references = REF_df.shape[0]
print("Número total de referencias ", known_references)

Número total de referencias  15


## Creando el BOM que aparece en el Excel

In [302]:
#BILL OF MATERIALS
BOM = np.zeros((known_references,known_references))
for index, row in df.iterrows():
  BOM[REF[row["Ref Salida"]],REF[row["Ref Entrada"]]] = float(row["Cantidad"].replace(',', '.'))
BOM_df = pd.DataFrame(BOM,index=REF_df, columns=REF_df)
print (BOM_df)

     AAA   AMA  BAJ  AAB  CAT  AAE  BAC  CAR  AAF  BAD  CAV  AAC  CAB  CAX  \
AAA  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AMA  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
BAJ  1.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AAB  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
CAT  0.0  0.07  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AAE  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
BAC  0.0  0.00  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
CAR  0.0  0.08  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AAF  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
BAD  0.0  0.00  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
CAV  0.0  0.05  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
AAC  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [303]:
BOM = BOM_df.values
mask_compras = BOM.sum(axis=0) == 0
mask_cliente = BOM.sum(axis=1) == 0

## Obtención de la demanda de cliente

In [304]:
DC_worksheet = AAM_workbook.worksheet('DC') #Convert to a DataFrame
rows = DC_worksheet.get_all_values()#Convert to a DataFrame
DC_df = pd.DataFrame(rows)
DC_df.iloc[0][0] = None
# Set row 1 as column names
DC_df.columns = DC_df.iloc[0]

# Set column 1 as row names
DC_df = DC_df.set_index(DC_df.iloc[:, 0])

# Remove row 1 and column 1
DC_df = DC_df.iloc[1:, 1:]
print(DC_df)
#Demandas de cliente
Profundidad = 15
DC = np.zeros((known_references,Profundidad))
for index, row in DC_df.iterrows():
  DC[REF[index],0] = row["Total"]
print('\n')
#print (DC)

0   Total Piezas/caja
CAB   560            
CAX  1800            
CAH  5200            




##Descargamos los datos de almacen las referencias



In [305]:
Almacen_worksheet = AAM_workbook.worksheet('ALMACEN') #Convert to a DataFrame
rows = Almacen_worksheet.get_all_values()#Convert to a DataFrame
Almacen_df = pd.DataFrame(rows)
Almacen_df.iloc[0][0] = None
# Set row 1 as column names
Almacen_df.columns = Almacen_df.iloc[0]

# Set column 1 as row names
Almacen_df = Almacen_df.set_index(Almacen_df.iloc[:, 0])

# Remove row 1 and column 1
Almacen_df = Almacen_df.iloc[1:, 1:]
print(Almacen_df)
#Demandas de cliente

0   STOCK Piezas caja €/UD Stock minimo Stock maximo Stock stop Lote max  \
AAA  2000          16   15          240          240        240     1000   
AAB  2000          20   23          240          240        240     1000   
AAC  2000          18   50          240          240        240     1000   
AAE  2000          18   44          240          240        240     1000   
AAF  2000          16   15          240          240        240     1000   
AMA  2000          20   98          240          240        240     1000   
CAB    32          32   71          240          240        240      704   
CAH    36          36   30          240          240        240      720   
CAX    20          20   33          240          240        240      720   
BAC    40          18   44           88          220        352      720   
BAD     0          32   15           30           75        120      704   
BAJ    48          16   98          196          490        784      720   
CAR     0   

In [306]:
Almacen = np.zeros((known_references, 9))
for index, row in Almacen_df.iterrows():
  Almacen[REF[index],0] = row["STOCK"]
  Almacen[REF[index],1] = row["Piezas caja"]
  Almacen[REF[index],2] = row["€/UD"]
  Almacen[REF[index],3] = row["Stock minimo"]
  Almacen[REF[index],4] = row["Stock maximo"]
  Almacen[REF[index],5] = row["Stock stop"]
  Almacen[REF[index],6] = row["Lote max"]
  Almacen[REF[index],7] = row["Lote min"]
  Almacen[REF[index],8] = row["T.CICLO"]
print(Almacen)

[[2000.   16.   15.  240.  240.  240. 1000. 1000.    0.]
 [2000.   20.   98.  240.  240.  240. 1000. 1000.    0.]
 [  48.   16.   98.  196.  490.  784.  720. 1440.   55.]
 [2000.   20.   23.  240.  240.  240. 1000. 1000.    0.]
 [  32.   20.   30.   60.  150.  240.  720. 1440.   33.]
 [2000.   18.   44.  240.  240.  240. 1000. 1000.    0.]
 [  40.   18.   44.   88.  220.  352.  720. 1440.   50.]
 [   0.   16.   71.  142.  355.  568.  720. 1440.   40.]
 [2000.   16.   15.  240.  240.  240. 1000. 1000.    0.]
 [   0.   32.   15.   30.   75.  120.  704. 1440.   60.]
 [  36.   60.   33.   66.  165.  264.  720. 1440.   40.]
 [2000.   18.   50.  240.  240.  240. 1000. 1000.    0.]
 [  32.   32.   71.  240.  240.  240.  704. 1420.  120.]
 [  20.   20.   33.  240.  240.  240.  720. 1419.  120.]
 [  36.   36.   30.  240.  240.  240.  720. 1440.  120.]]


Extraemos la informacion de las siguientes matrices:

Stock: Strock inicial

BOX: Piezs por caja

Prices: Coste de carda referencia

Matrix_Decision: matriz para la toma de decision de qué es lo que producimos

T: Tiempos de ciclo

In [307]:
#extraigo el stock inicial
Stock_ini = np.expand_dims(np.copy(Almacen[:,0]), axis = 0)
#Extraigo las piezas por caja
BOX = np.expand_dims(np.copy(Almacen[:,1]), axis = 0)
#extraigo los precios de cada referencia
Prices = np.expand_dims(np.copy(Almacen[:,2]), axis = 0)
#Extraigo la matriz de decisiones
Matrix_decision = np.copy(Almacen[:,3:7])
#Extraigo los tiempos de ciclo
T = np.expand_dims(np.copy(Almacen[:,8]), axis = 0)
T_current = np.expand_dims(np.copy(Almacen[:,8]), axis = 0)
T_current [0,mask_compras] = -1


Simulacion de un BOM con dos productos diferentes

**Listado de Maquinaria**

mumMaquinas = número de máquinas

DICMAQ = Diccionario de maquinaria (m x 1)

BOMMAQ = Listado de maquinaria por referencia (m x n)

##Descargamos los datos de las máquinas

In [308]:
Maquinas_worksheet = AAM_workbook.worksheet('MAQUINARIA') #Convert to a DataFrame
rows = Maquinas_worksheet.get_all_values()#Convert to a DataFrame
MAQUINAS_df = pd.DataFrame(rows)
# Set row 1 as column names
MAQUINAS_df.columns = MAQUINAS_df.iloc[0]
# Remove row 1 and column 1
MAQUINAS_df = MAQUINAS_df.iloc[1:,:]
print(MAQUINAS_df)


0 COD. MAQUINA POTENCIA
1          ENC        0
2         MEC1      125
3         MEC2      100
4         VUL1      150
5         VUL2      200


##Creamos el diccionario de las máquinas

In [309]:
DICMAQ = {"Compras":0}
MAQ_df = MAQUINAS_df['COD. MAQUINA'].drop_duplicates().sort_values()
print ("dataframe de Maquinas")
print(MAQ_df)
n = 1
for i in MAQ_df:
  DICMAQ[i] = n
  n += 1
print('\n')
print("Diccionario de máquinas")
print(DICMAQ)

numMaquinas = MAQ_df.shape[0] + 1
print ("Número de máquinas" , numMaquinas)



dataframe de Maquinas
1     ENC
2    MEC1
3    MEC2
4    VUL1
5    VUL2
Name: COD. MAQUINA, dtype: object


Diccionario de máquinas
{'Compras': 0, 'ENC': 1, 'MEC1': 2, 'MEC2': 3, 'VUL1': 4, 'VUL2': 5}
Número de máquinas 6


## Creamos el BOM de la maquinaria (Referencia vs Maquina)

In [310]:
Procesos_worksheet = AAM_workbook.worksheet('PROCESOS') #Convert to a DataFrame
rows = Procesos_worksheet.get_all_values()#Convert to a DataFrame
Procesos_df = pd.DataFrame(rows)
# Set row 1 as column names
Procesos_df.columns = Procesos_df.iloc[0]
# Remove row 1 and column 1
Procesos_df = Procesos_df.iloc[1:,:]
print(Procesos_df)

0  Ref Salida PROCESO TIEMPO CICLO
1         CAT     ENC           20
2         CAR     ENC           16
3         CAV     ENC           25
4         BAD    MEC2           90
5         BAJ    MEC2          100
6         BAC    MEC2           80
7         BAD    MEC1           90
8         BAJ    MEC1          100
9         BAC    MEC1           80
10        CAB    VUL1           60
11        CAX    VUL1           50
12        CAH    VUL1           75
13        CAB    VUL2           60
14        CAX    VUL2           50
15        CAH    VUL2           75


In [311]:
BOMMAQ = np.zeros((numMaquinas,known_references))
for index, row in Procesos_df.iterrows():
  BOMMAQ[DICMAQ[row["PROCESO"]], REF[row["Ref Salida"]]] = 1
print (BOMMAQ)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]


##Calculamos las demandas internas

In [312]:
#Voy incremetnando la matriz de demandas a medida que aumenta el nicel de fabricación
def Calcula_necesidades():
  for profIndex in range(Profundidad-1):
    DC[:,profIndex+1] = np.dot(BOM.T,DC[:,profIndex])
  return(DC.sum(axis = 1))

In [313]:
DC_internas = Calcula_necesidades()
print("Demandas para el día", '\n')
print (DC_internas)

Demandas para el día 

[1120.   482.4 1120.  1720.  1120.  7000.  1800.  1800.  1800.  5200.
 5200.  4680.   560.  1800.  5200. ]


In [318]:
mask_maquina = BOMMAQ[2,:] == 1

In [319]:
#Funcion de toma de decisión de qué maquina ha de trabajar
def Making_decision():
  Matrix_necessity = DC_internas - Stock_ini
  Matrix_necessity[mask_compras] = 0
  for i in range(0,numMaquinas):
    mask_maquina = BOMMAQ[i,:] == 1

  aux_array = np.multiply(np.ones(Matrix_necessity.shape).T, Stock_ini)
  mask_necesity = Matrix_decision > aux_array.T
  Production_array = np.copy(Stock_ini)
  Production_array[0,np.logical_not(mask_necesity[:,0])] = 0
  return(Production_array)

In [ ]:
Vector_produccion = Making_decision()
print (Vector_produccion)


In [ ]:
def Actualiza_stock(mask_finished_part, Stock_ini):
  produced = np.zeros((known_references,1))
  produced[mask_finished_part.T] = 1
  Stock_ini = Stock_ini + produced.T
  Stock_ini = Stock_ini - np.dot(BOM.T, produced).T
  T_current[mask_finished_part] = T[mask_finished_part]
  return(Stock_ini)

In [ ]:
#Funcion para el cálculo de piezas producidas en un tiempo determinado en segundos.
def Produce( seconds, T_current,  Stock_ini):
  for i in range(0,seconds+1):
    T_current = T_current - np.ones(known_references)
    mask_finished_part = T_current == np.zeros((known_references))
    if (mask_finished_part.any() == True):
      Stock_ini = Actualiza_stock(mask_finished_part, Stock_ini)
      #if np.any(Stock_ini[mask_finished_part] % BOX[mask_finished_part]) == 0:
  return(T_current, Stock_ini)


In [ ]:
Stock_ini = np.expand_dims(np.copy(Almacen[:,0]), axis = 0)
T_current = np.copy(T)
print (T_current)
print(Stock_ini)
T_current, Stock_ini = Produce(35, T_current, Stock_ini)
print ("salida")
print(Stock_ini)
print (T_current)


In [ ]:
#Busco en número de cajas necesario para la produccion
DCBOX = np.copy(DC);
for dayIndex in range(Dias):
  mask = np.mod(DC[:,:,dayIndex].T, BOX) > 0
  ones_array = np.zeros_like(np.mod(DC[:,:,dayIndex], BOX.T))
  ones_array [mask.T] = 1
  DCBOX[:,:,dayIndex] = np.floor_divide(DC[:,:,dayIndex], BOX.T) +  ones_array
  print (DCBOX[:,:,dayIndex],'\n')



MAQ = is a vector of ONES where there is an operation


In [ ]:
MAQ = np.zeros((numMaquinas,Dias));
for dayIndex in range(Dias):
  MAQ[:,dayIndex] = np.dot(DCHORAS[:,:,dayIndex].sum(axis = 1), BOMMAQ.T)
  print(MAQ[:,dayIndex],'\n')

In [ ]:
# Calculo el numero de cajas para cada referencia
Cajas_stock_global = np.divide(Stock_ini , BOX)
print (Cajas_stock_global)

##Separo stock por almacenes

In [ ]:
Sim_almacen = np.ones((known_references,3))
mask1 = np.logical_not (np.sum(BOM, axis = 1) == 0)
mask3 = np.logical_not (np.sum(BOM, axis = 0) == 0)
mask2 = np.logical_not (mask1 & mask3)
Sim_almacen = np.multiply(Sim_almacen, Cajas_stock_global)
Sim_almacen [mask1,0] = 0
Sim_almacen [mask2,1] = 0
Sim_almacen [mask3,2] = 0
print (Sim_almacen)

In [ ]:
#Comparo el stock de cajas con la capacidad del almacen
Capa_almac_worksheet = AAM_workbook.worksheet('CAPACIDAD ALMACEN') #Convert to a DataFrame
rows = Capa_almac_worksheet.get_all_values()#Convert to a DataFrame
Capa_almac_df = pd.DataFrame(rows)
# Set row 1 as column names
Capa_almac_df.columns = Capa_almac_df.iloc[0]
# Remove row 1 and column 1
Capa_almac_df = Capa_almac_df.iloc[1:,:]
print(Capa_almac_df)

In [ ]:
Capa_alm = np.zeros((3,1))
for index, row in Capa_almac_df.iterrows():
  Capa_alm[index - 1] = row["CANTIDAD"]
print (Capa_alm)
print(np.sum(Sim_almacen,axis = 0))
Almacen_capaz = np.all(np.sum(Sim_almacen,axis = 0) < Capa_alm)
print(" El almacen es capaz", Almacen_capaz)

# Definición de Bases de la Simulación
## Constraints de la simulacion

LoteMin = Lote mínimo de producción (n x 1)

LotMax = Lote máximo de producción (n x 1)

TCambio = Tiempos de cambio de referencia (n x n)

Planif =  Planificacion de la producción (Maq x 24 horas x Ref)

PrioMat = Matriz de priodidades (n x 1)

*Donde n = known_references*

In [ ]:
Transiciones_worksheet = AAM_workbook.worksheet('TRANSICIONES') #Convert to a DataFrame
rows = Transiciones_worksheet.get_all_values()#Convert to a DataFrame
Transiciones_df = pd.DataFrame(rows)
# Set row 1 as column names
Transiciones_df.columns = Transiciones_df.iloc[0]
# Remove row 1 and column 1
Transiciones_df = Transiciones_df.iloc[1:,:]
print(Transiciones_df)


In [ ]:
Transiciones = np.zeros((numMaquinas,known_references, known_references))
for index, row in Transiciones_df.iterrows():
  Transiciones[DICMAQ[row["PROCESO"]], REF[row["Origen"]], REF[row["Destino"]]] =  row["Tiempo"]
for key in DICMAQ.keys():
  print ("Tiempos de transición para la máquina ", key, '\n')
  print (Transiciones[DICMAQ[key],:,:])

In [ ]:
print (MAQ)
print (DCHORAS[:,:,0])

# Cálculo de costes del proceso

En esta sección calcularemos los costes del proceso.

En esencia el cálculo es el siguiente:

Coste proceso [CP] = Coste mano de obra[CMO] + Coste Máquina [CMQ]

Desarrollando un poco mas este cálculo tenemos que:

 CMO = Coste operario [MO] / Tiempo de ciclo [T] / 3600

CMQ = Coste Máquina fijo [CMQF] Coste Máquina Variable  [CMQV]

CMQF = Coste fijo Máquina [CF] * T / 3600

CMQV = Coste potencia [CPOT] * Potencia Máquina [POT] * T / 3600

Sustituyento tenemos:

CP = (MO + CF + CPOT * POT) * T / 3600

Ahora nos centraremso en el cálculo de la mano de obra

MAQREF = Vector Maquinas (n x 1)

MOM = Coste de la mano de obra Turno de mañana

MOT = Coste de la mano de obra Turno de tarde

MON = Coste de la mano de obra Turno de noche

MHORARIA = MATRIZ Horaria (n x 24)

NOP =  Número de operarios (n x 1) (Dato a recibir de cliente)

CMOTHORARIO = Coste de mano de obra total (nx24) resultado de MOTURNOS * MOP *T / 3600

In [ ]:
MAQREF = np.zeros((known_references,1))
mask = BOMMAQ.sum(axis = 0) > 0
MAQREF[mask] = 1
print(MAQREF)
MHORARIA = np.ones((known_references,24))
MOTURNOS = np.multiply(MHORARIA, MAQREF)
print(MOTURNOS)

In [ ]:
MOM = 12
MOT = 14
MON = 18
MOTURNOS [:,0:6] *=  MON
MOTURNOS [:,6:14] *=  MOM
MOTURNOS [:,14:22] *=  MOT
MOTURNOS [:,22:24] *=  MON
print(MOTURNOS)

In [ ]:
CMOTHORARIO  =np.multiply(MOTURNOS, np.multiply(T,NOP)) / 3600
print(CMOTHORARIO)


Ahora nos centraremso en el cálculo de la mano de obra

MAQ = Vector Maquinas (n x 1)

CMF = Coste máquina fijo (n x 24)

CPOT = Tabla de coste de potencias (n x 24)

POT = Potencia de Maquinaria (n x 1) KWH

CELECM = Coste potencia turno mañana

CELECT = coste potencia turno tarde

CELECN = Coste potencia turno Noche

CMVTURNOS = Coste maquina variable (n x 24))

T =  Tiempos de ciclo (n x 1) (Definidos anteriormente)

CMQVTHORARIO = Coste de maquina total (nx24) resultado de CMVTURNOS * POT *T / 3600

CMTOT = (CMF + CMQVTHORARIO) * T / 3600

In [ ]:
from genericpath import commonprefix
#Coste maquina definido por cliente CMF
cmfijo = 5  #5 es un valor estimado
CMF = np.ones((known_references,24))
CMF = np.multiply(CMF, MAQREF)
CMF = CMF * cmfijo
print (CMF)

In [ ]:
CMTURNOS = np.ones((known_references,24))
CMTURNOS = np.multiply(CMTURNOS, MAQREF)
print(CMTURNOS)

In [ ]:

CELECM = 0.12
CELECT = 0.14
CELECN = 0.16
CMTURNOS [:,0:6] *=  CELECN
CMTURNOS [:,6:14] *=  CELECM
CMTURNOS [:,14:22] *=  CELECT
CMTURNOS [:,22:24] *=  CELECN
print(CMTURNOS)

In [ ]:
POT_referencia = np.multiply(BOMMAQ,CPOT)
POT_referencia = np.expand_dims(POT_referencia.sum(axis = 0), axis = 0)

print(POT_referencia)

In [ ]:
CMQVTHORARIO  = np.multiply(CMTURNOS , POT_referencia.T)
print(CMQVTHORARIO)

In [ ]:
CMTOT = np.multiply((CMF + CMQVTHORARIO), T) / 3600
print (CMTOT)

El coste total es el resultado de sumar el coste máquina y el de operario
CTOTHORARIO

In [ ]:
CTOTHORARIO = CMOTHORARIO + CMTOT
print (CTOTHORARIO)

# **Inicio de modelado de simulación**
El modelo de la simulación requiere que se definan:
1. Descipción de los componentes del sistema (ver zona anterior)
2. El reloj de la simulación (arreglo de tiempos) se considera trabajar por semanas de 6 días 0-lunes -- 5 sábado
3. Definición del estado inicial del sistema
4. El modelo para el cambio de estado
5. Procedimiento para adelantar el reloj
Como resultado del sistema se provee:
1. Condición inicial
2. Listado de eventos (aleatorios + paramétricos)
3. Resultado obtenido de la ejecución en parámetros evaluables de la efectividad del resultado.

*Reloj de la simulación* - se establece como una matriz horarira de 24 x 6 días (programación semanal)

In [ ]:
Calendario_Sim = np.zeros((24,6))
print(Calendario_Sim)

*Definir el estado inicial* - Estado de las máquinas y almacenamiento

Se definie un Coste de referencia para el calculo del coste de material en el almacén

In [ ]:
Process_status = np.zeros((len(DICMAQ),1))
Process_status[DICMAQ["Inyeccion"]] = REF["5"]
Process_status[DICMAQ["Mecanizado"]] = REF["8"]
STOCK = {"Referencia":0,"Volumen":1,"Coste":2}
Reference_cost = np.zeros((len(REF),1))
Reference_cost[REF["1"]] = 12.8
Reference_cost[REF["2"]] = 23.7
Reference_cost[REF["3"]] = 18.4
Reference_cost[REF["4"]] = 6.2
Reference_cost[REF["5"]] = 50.1
Reference_cost[REF["6"]] = 80.2
Reference_cost[REF["7"]] = 67.4
Reference_cost[REF["8"]] = 73.9

Almacen_MP = np.zeros((3,known_references))
Almacen_MP[STOCK["Referencia"],REF["1"]] = REF["1"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["1"]] = 5
Almacen_MP[STOCK["Coste"],REF["1"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["1"]]

Almacen_MP[STOCK["Referencia"],REF["2"]] = REF["2"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["2"]] = 7
Almacen_MP[STOCK["Coste"],REF["2"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["2"]]

Almacen_MP[STOCK["Referencia"],REF["3"]] = REF["3"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["3"]] = 1
Almacen_MP[STOCK["Coste"],REF["3"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["3"]]

Almacen_MP[STOCK["Referencia"],REF["4"]] = REF["4"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["4"]] = 2
Almacen_MP[STOCK["Coste"],REF["4"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["4"]]


# Notas Alf - Borrar
#MAQ_TRANS_TIEMPO[2,TRANS["Maquina"]] = DICMAQ["Inyeccion"] / "Mecanizado"
#MAQ_TRANS_TIEMPO[2,TRANS["Origen"]] = REF["5"]             / REF["7"]
#MAQ_TRANS_TIEMPO[2,TRANS["Destino"]] = REF["6"]            / REF["8"]

print("Reference costs:",Reference_cost,"\n")
print("Current status of RowMat Stock\n",Almacen_MP)
print("Process Current State: ",Process_status)

In [ ]:
sim_start = dt.datetime(year=2023, month=6, day=23, hour=0, minute=0, second=0)
step = timedelta(minutes=15)
sim_frame = sim_start
sim_log = []
for i in range(0,12):
  print("Frame [",i,"] ",sim_frame)
  sim_frame += step
  sim_log.append(sim_frame)

print("SIM Log:\n",sim_log)